In [10]:
import pandas as pd
df3 = pd.read_csv("mul_anomaly.csv")
df3.head(10)


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Anomaly
0,Male,No,0,Graduate,No,5849,0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0,141.0,360.0,1.0,Urban,Y
5,Male,Yes,2,Graduate,Yes,5417,4196,267.0,360.0,1.0,Urban,Y
6,Male,Yes,0,Not Graduate,No,2333,1516,95.0,360.0,1.0,Urban,Y
7,Male,Yes,3+,Graduate,No,3036,2504,158.0,360.0,0.0,Semiurban,N
8,Male,Yes,2,Graduate,No,4006,1526,168.0,360.0,1.0,Urban,Y
9,Male,Yes,1,Graduate,No,12841,10968,349.0,360.0,1.0,Semiurban,N


In [11]:

###########  Missing Value Treatment  #########

for i in df3.columns:
  clas = df3[i].dtypes
  if clas == 'object':
    df3[i].fillna(df3[i].mode()[0], inplace=True)
  else:
    df3[i].fillna(df3[i].mean(), inplace=True)

df3.dtypes

Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome      int64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Anomaly               object
dtype: object

In [13]:
##########  Applying Label encoder for converting data type as objects to integer  ##########
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df4 = df3.copy()
for i in df4.columns:
  cls = df4[i].dtypes
  if cls == 'object':
    df4[i] = df4[[i]].astype(str).apply(le.fit_transform)
  else:
    df4[i] = df4[i]

df4.head(13)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Anomaly
0,1,0,0,0,0,5849,0,134.255319,360.0,1.0,2,1
1,1,1,1,0,0,4583,1508,128.000000,360.0,1.0,0,0
2,1,1,0,0,1,3000,0,66.000000,360.0,1.0,2,1
3,1,1,0,1,0,2583,2358,120.000000,360.0,1.0,2,1
4,1,0,0,0,0,6000,0,141.000000,360.0,1.0,2,1
5,1,1,2,0,1,5417,4196,267.000000,360.0,1.0,2,1
6,1,1,0,1,0,2333,1516,95.000000,360.0,1.0,2,1
7,1,1,3,0,0,3036,2504,158.000000,360.0,0.0,1,0
8,1,1,2,0,0,4006,1526,168.000000,360.0,1.0,2,1
9,1,1,1,0,0,12841,10968,349.000000,360.0,1.0,1,0


In [14]:
for i in df4.columns:
  print(sum(pd.isnull(df4[i])))

0
0
0
0
0
0
0
0
0
0
0
0


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve

#def find_anomaly(dataframe, output_variable):
X = df4.drop('Anomaly', 1)  ## Defining X and Y variables
y = df4['Anomaly']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/4, random_state=42)  ### Splitting train and test datasets

classifier =  LogisticRegression()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
y_predd = ["Anomaly" if i == 1 else "Not Anomaly" for i in y_pred]

cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 4  5]
 [ 0 16]]


In [6]:
print('Accuracy: ' + str(accuracy_score(y_test, y_pred)))


Accuracy: 0.84


In [18]:
# predict probabilities
lr_probs_RF = classifier.predict_proba(X_test)
# keep probabilities for the positive outcome only
lr_probs_RF = lr_probs_RF[:, 1]


In [19]:
lr_precision_RF, lr_recall_RF, _ = precision_recall_curve(y_test, lr_probs_RF)
lr_f1_RF, lr_auc_RF = f1_score(y_test, y_pred), auc(lr_recall_RF, lr_precision_RF)
# summarize scores
print('Random Forest: f1=%.3f auc=%.3f' % (lr_f1_RF, lr_auc_RF))

Random Forest: f1=0.865 auc=0.896


In [20]:
#return (y_pred, max(lr_probs_RF))

X_test['Prediction'] = y_predd
X_test

C:\Users\msurisetty3\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Prediction
62,1,1,0,1,1,2609,3449,165.000000,180.000000,0.000000,0,Not Anomaly
40,1,0,0,0,0,3600,0,80.000000,360.000000,1.000000,2,Anomaly
95,1,0,0,0,0,6782,0,134.255319,360.000000,0.835165,2,Anomaly
18,1,1,0,1,0,4887,0,133.000000,360.000000,1.000000,0,Anomaly
97,1,1,0,0,0,1977,997,50.000000,360.000000,1.000000,1,Anomaly
84,1,1,1,0,0,3988,0,50.000000,240.000000,1.000000,2,Anomaly
64,0,0,0,0,0,4166,0,116.000000,360.000000,0.000000,1,Not Anomaly
42,1,1,0,0,0,2400,0,75.000000,360.000000,0.835165,2,Anomaly
10,1,1,2,0,0,3200,700,70.000000,360.000000,1.000000,2,Anomaly
0,1,0,0,0,0,5849,0,134.255319,360.000000,1.000000,2,Anomaly
